In [2]:
import os
import sys
from pathlib import Path

CURRENT_DIRECTORY = Path(os.getcwd())
ROOT_DIRECTORY = (CURRENT_DIRECTORY / "..").absolute().resolve()

print(f"Current directory: {CURRENT_DIRECTORY}")
print(f"Root directory: {ROOT_DIRECTORY}")

sys.path.append(str(ROOT_DIRECTORY))

Current directory: /home/ubuntu/arga-arc/tf_coder
Root directory: /home/ubuntu/arga-arc


In [3]:
import typing as t
import json
from pprint import pprint
from dataclasses import dataclass
import tensorflow as tf
import numpy as np
import math
from config import CONFIG
from openai import OpenAI
import re
from collections import Counter
import random
import ast
import traceback
from tf_coder.utils import Example

pprint(CONFIG.__dict__.keys())

OPENAI = OpenAI(api_key=CONFIG.OPENAI_SECRET_KEY, organization=CONFIG.OPENAI_ORGANIZATION)

2024-05-01 17:38:36.095506: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-01 17:38:36.098600: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-01 17:38:36.137310: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-01 17:38:36.137337: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-01 17:38:36.138189: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

Current directory: /home/ubuntu/arga-arc/tf_coder
Root directory: /home/ubuntu/arga-arc
TF Operators: 111, ['tf.abs(x)', 'tf.add(x, y)', 'tf.add_n(inputs)', 'tf.argmax(input, axis)', 'tf.argmin(input, axis)']
SparseTF Operators: 19, ['tf.SparseTensor(indices, values, dense_shape)', 'tf.sparse.add(a, b)', 'tf.sparse.concat(axis, sp_inputs)', 'tf.sparse.expand_dims(sp_input, axis)', 'tf.sparse.from_dense(tensor)']
dict_keys(['OPENAI_SECRET_KEY', 'OPENAI_ORGANIZATION', 'TOGETHER_SECRET_KEY', 'TOGETHER_BASE_URL', 'OCTO_SECRET_KEY'])


In [3]:
OUTPUTS = CURRENT_DIRECTORY / "tfcoder_dataset_with_completions.in_context_ordered_by_weight.json"
OUTPUTS_JSON = json.loads(OUTPUTS.read_text())

pprint(OUTPUTS_JSON[0])

{'completions': ['return tf.stack([tf.math.bincount(in1), in1], axis=1)',
                 'return tf.stack([tf.divide(in1, tf.shape(in1)[0]), '
                 'tf.mod(in1, tf.shape(in1)[0])], axis=1)',
                 'return tf.stack([tf.math.floor_div(in1, tf.constant(2, '
                 'dtype=tf.int32)), tf.math.floormod(in1, tf.constant(2, '
                 'dtype=tf.int32))], axis=1)',
                 'return tf.stack([tf.floor_div(in1, 2), tf.mod(in1, 2)], '
                 'axis=1)',
                 'return tf.stack([tf.math.floor_divide(in1, '
                 'tf.shape(in1)[0]), tf.math.mod(in1, tf.shape(in1)[0])], '
                 'axis=1)',
                 'return tf.stack([tf.math.floor_divide(in1, '
                 'tf.reduce_max(in1) + 1), tf.math.floormod(in1, '
                 'tf.reduce_max(in1) + 1)], axis=1)',
                 'return tf.stack([tf.math.floor_div(in1, tf.reduce_max(in1) + '
                 '1), tf.math.floormod(in1, tf.reduce_max(in1)

In [4]:
EXAMPLES = [Example.from_json(output["examples"]) for output in OUTPUTS_JSON]


for example in EXAMPLES[:5]:
    pprint(example)
    print(example.max_input_rank)
    print(example.dimension_lengths)


Example(inputs=[array([0, 0, 0, 1, 3, 3])],
        output=array([[0, 0],
       [0, 1],
       [0, 2],
       [1, 0],
       [3, 0],
       [3, 1]]),
        input_names=None)


2024-04-03 21:05:05.102097: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


TypeError: 'NoneType' object is not iterable

In [ ]:
COMPLETIONSES = [
    output['response']['completions'] if 'response' in output else None 
    for output in OUTPUTS_JSON
]

print(len(COMPLETIONSES))
pprint(COMPLETIONSES[13])

72
['return tf.roll(in1, shift=1, axis=1)',
 'return tf.roll(in1, shift=1, axis=1)',
 'return tf.roll(in1, shift=1, axis=1)',
 'return tf.roll(in1, shift=1, axis=1)',
 'return tf.roll(in1, shift=1, axis=1)',
 'return tf.roll(in1, shift=1, axis=1)',
 'return tf.roll(in1, shift=1, axis=1)',
 'return tf.roll(in1, shift=1, axis=1)',
 'return tf.roll(in1, shift=1, axis=1)',
 'return tf.roll(in1, shift=-1, axis=-1)']


## parse constants

In [ ]:
TEST_COMPLETIONS = [
    COMPLETIONSES[0][0],
    "return tf.pad(in1, [[0,0],[0,1]])",
    "return tf.gather(in1, in2, batch_dims=1)",
    "indices = tf.stack([tf.range(tf.shape(in2)[0]), in2], axis=1)\nreturn tf.gather_nd(in1, indices)",
    "return tf.cast(tf.sequence_mask(in1, maxlen=tf.reduce_max(in1)), tf.int32)",
    "return tf.where(in1, in2, tf.multiply(in2, tf.constant(-10)))",
    "return tf.tensor_scatter_nd_update(in1['tensor'], in1['indices'], in1['updates'])",
    "return tf.reduce_any(tf.one_hot(in1, 9), axis=1, keepdims=False)",
]
TEST_ASTS = [ast.parse(completion) for completion in TEST_COMPLETIONS]

In [25]:
TEST_NORMALIZE_COMPLETIONS = [
    """```python                                                                         
return tf.reduce_any(in1, axis=1)
```""",
    """```python                                                                           
def transform(in1):
    indices = tf.expand_dims(tf.range(tf.size(in1)), axis=1)
    updates = tf.expand_dims(in1, axis=1)
    return tf.concat([indices, updates], axis=1)
```""",
"""```python                                                                           
def transform(in1, in2):
    unique_vals, idx = tf.unique(in1)
    return tf.math.segment_max(in2, idx)
```""",
"""```python                                                                           
def transform(in1, in2):
    import tensorflow as tf
    input_tensor = tf.stack([in1, in2], axis=1)
    max_values = tf.reduce_max(input_tensor, axis=0)
    return max_values
```""",
"""```python                                                                           
return tf.gather(tf.gather_nd(in1, tf.argsort(tf.stack([in1[:,0], in1[:,1]], axis=1))), tf.argsort(tf.stack([in1[:,0], in1[:,1]], axis=1)[:,1])
```""",
"""```python                                                                           
distances = []
for i in range(len(in1)):
    distances.append(tf.reduce_min(tf.sqrt(tf.reduce_sum(tf.square(in1[i] - in2), axis=1))))
return distances
```""",
"""```python                                                                           
    def transform(in1):
        import tensorflow as tf
        mat = tf.zeros([5, 6], dtype=tf.int32)
        index = in1
        mat = tf.tensor_scatter_nd_update(mat, [[i, index] for i in range(5)], tf.ones(5, dtype=tf.int32))
        return mat
```"""
]


CODE_BLOCK_REGEX = r".*```(?:[\w\s]+)?\n(.*?)\n```.*"
CODE_LINE_REGEX = r".*`(.*)`.*"
DEF_LINE_REGEX = r"\s*def\s+\w+\(.*\):"
RETURN_REGEX = r"([\s]+)return"
ASSIGN_REGEX = r"([\s]+)([a-zA-Z_][a-zA-Z0-9_\s,]*)([\s]+)="
IMPORT_REGEX = r"([\s]+)import"
CATCHALL_REGEX = r"^([\s]+)(.*)"


def normalize_completion(completion: str) -> str:
    match = re.match(CODE_BLOCK_REGEX, completion, re.DOTALL)
    if match:
        completion = match.group(1)
    else:
        match = re.match(CODE_LINE_REGEX, completion)
        if match:
            completion = match.group(1)
    
    try:
        ast.parse(completion)
        return completion
    except SyntaxError:
        pass

    lines = completion.split("\n")
    if re.match(DEF_LINE_REGEX, lines[0]):
        return completion

    normalized_lines = []
    for line in lines:
        normalized_line = re.sub(RETURN_REGEX, "return", line)
        normalized_line = re.sub(ASSIGN_REGEX, r"\2 =", normalized_line)
        normalized_line = re.sub(IMPORT_REGEX, "import", normalized_line)
        normalized_line = re.sub(CATCHALL_REGEX, r"\2", normalized_line)
        normalized_lines.append(normalized_line)
    normalized_completion = "\n".join(normalized_lines)
    return normalized_completion

for completion in TEST_NORMALIZE_COMPLETIONS:
    print('completion:',completion)
    print('normalized:',normalize_completion(completion))
    print()

completion: ```python                                                                         
return tf.reduce_any(in1, axis=1)
```
normalized: return tf.reduce_any(in1, axis=1)

completion: ```python                                                                           
def transform(in1):
    indices = tf.expand_dims(tf.range(tf.size(in1)), axis=1)
    updates = tf.expand_dims(in1, axis=1)
    return tf.concat([indices, updates], axis=1)
```
normalized: def transform(in1):
    indices = tf.expand_dims(tf.range(tf.size(in1)), axis=1)
    updates = tf.expand_dims(in1, axis=1)
    return tf.concat([indices, updates], axis=1)

completion: ```python                                                                           
def transform(in1, in2):
    unique_vals, idx = tf.unique(in1)
    return tf.math.segment_max(in2, idx)
```
normalized: def transform(in1, in2):
    unique_vals, idx = tf.unique(in1)
    return tf.math.segment_max(in2, idx)

completion: ```python                   

In [ ]:
class ConstantVisitor(ast.NodeVisitor):
    def __init__(self):
        self.constants = []
    
    def visit_Constant(self, node: ast.Constant):
        if type(node.value) == int:
            self.constants.append(node.value)

    def visit_UnaryOp(self, node: ast.UnaryOp):
        if isinstance(node.op, ast.USub) and isinstance(node.operand, ast.Constant):
            self.constants.append(-node.operand.value)
        else:
            self.generic_visit(node)

def get_constants(ast_node: ast.Module) -> t.List[str]:
    visitor = ConstantVisitor()
    visitor.visit(ast_node)
    return visitor.constants

for completion, ast_ in zip(TEST_COMPLETIONS, TEST_ASTS):
    print(completion)
    print(get_constants(ast_))
    print()


return tf.stack([tf.math.bincount(in1), in1], axis=1)
[1]

return tf.pad(in1, [[0,0],[0,1]])
[0, 0, 0, 1]

return tf.gather(in1, in2, batch_dims=1)
[1]

indices = tf.stack([tf.range(tf.shape(in2)[0]), in2], axis=1)
return tf.gather_nd(in1, indices)
[0, 1]

return tf.cast(tf.sequence_mask(in1, maxlen=tf.reduce_max(in1)), tf.int32)
[]

return tf.where(in1, in2, tf.multiply(in2, tf.constant(-10)))
[-10]

return tf.tensor_scatter_nd_update(in1['tensor'], in1['indices'], in1['updates'])
[]

return tf.reduce_any(tf.one_hot(in1, 9), axis=1, keepdims=False)
[9, 1]



In [ ]:
ASTSES = []

for completions in COMPLETIONSES:
    if completions is None:
        ASTSES.append(None)
        continue
    
    asts = []
    for completion in completions:
        try:
            asts.append(ast.parse(completion))
            continue
        except SyntaxError as e:
            pass

        normalized_completion = normalize_completion(completion) 
        try:
            asts.append(ast.parse(normalized_completion))
        except SyntaxError as e:
            print("# completion")
            print(completion)
            print()
            print("# normalized_completion")
            print(normalized_completion)
            print()
            print("# error")
            print(e)
            print()
            print()
            asts.append(None) 
        
    ASTSES.append(asts)

print(len(ASTSES))
pprint(ASTSES[0])

# completion
return tf.where(in1, in2, tf.multiply(-10, tf.range(tf.shape(in2)[0]));

# normalized_completion
return tf.where(in1, in2, tf.multiply(-10, tf.range(tf.shape(in2)[0]));

# error
invalid syntax (<unknown>, line 1)


72


In [ ]:
CONSTANTSES = [
    [
        get_constants(ast_node) if ast_node is not None else None 
        for ast_node in ast_nodes
    ] if ast_nodes is not None else None
    for ast_nodes in ASTSES 
]

print(len(CONSTANTSES))
pprint(CONSTANTSES[0])

72
[[1],
 [0, 0, 1],
 [2, 2, 1],
 [2, 2, 1],
 [0, 0, 1],
 [1, 1, 1],
 [1, 1, 1],
 [1, 1, 1],
 [1],
 [0, 1]]


In [ ]:
for output, constants in zip(OUTPUTS_JSON, CONSTANTSES):
    if 'response' not in output:
        continue
    output['response']['constants'] = constants
    output['response']['all_constants'] = list(set([
        constant 
        for constants in constants if constants is not None 
        for constant in constants
    ]))

pprint(OUTPUTS_JSON[0])

{'completions': ['return tf.stack([tf.math.bincount(in1), in1], axis=1)',
                 'return tf.stack([tf.divide(in1, tf.shape(in1)[0]), '
                 'tf.mod(in1, tf.shape(in1)[0])], axis=1)',
                 'return tf.stack([tf.math.floor_div(in1, tf.constant(2, '
                 'dtype=tf.int32)), tf.math.floormod(in1, tf.constant(2, '
                 'dtype=tf.int32))], axis=1)',
                 'return tf.stack([tf.floor_div(in1, 2), tf.mod(in1, 2)], '
                 'axis=1)',
                 'return tf.stack([tf.math.floor_divide(in1, '
                 'tf.shape(in1)[0]), tf.math.mod(in1, tf.shape(in1)[0])], '
                 'axis=1)',
                 'return tf.stack([tf.math.floor_divide(in1, '
                 'tf.reduce_max(in1) + 1), tf.math.floormod(in1, '
                 'tf.reduce_max(in1) + 1)], axis=1)',
                 'return tf.stack([tf.math.floor_div(in1, tf.reduce_max(in1) + '
                 '1), tf.math.floormod(in1, tf.reduce_max(in1)

In [ ]:
OUTPUT_FILE = CURRENT_DIRECTORY / "tfcoder_dataset_with_completions_and_constants.json"
OUTPUT_FILE.write_text(json.dumps(OUTPUTS_JSON, indent=4))

258065

## parse and count constants of different types

- common: 0, 1, -1, True, False
- axis: 2 -> max_rank
- shape: {dim. lens}
- provided: anything else
- tf.int32
- tf.float32
- tf.int64
- tf.bool
- input var: in1, in2, ...
- shape tuple: the shape of the output tuple (constant)

In [ ]:
TEST_COMPLETIONS = [
    COMPLETIONSES[0][0],
    COMPLETIONSES[2][0],
    "return tf.sparse.slice(in1, start=[3,800,2], size=[1,-1,-1])",
    COMPLETIONSES[3][0],
    "tf.reshape(in1, shape=(4, 3, 2))",
]
TEST_ASTS = [ast.parse(completion) for completion in TEST_COMPLETIONS]
TEST_EXAMPLES = [
    EXAMPLES[0],
    EXAMPLES[2],
    EXAMPLES[2],
    EXAMPLES[3],
    EXAMPLES[3]
]

In [ ]:
from ast import Attribute, Name
from typing import Any


class ConstantCounts(t.TypedDict):
    common: int
    axis: int
    shape: int
    provided: int
    tf_int32: int
    tf_float32: int
    tf_int64: int
    tf_bool: int
    input_var: int
    shape_tuple: int

def add_constant_counts(counts1: ConstantCounts, counts2: ConstantCounts) -> ConstantCounts:
    return {
        key: counts1.get(key, 0) + counts2.get(key, 0)
        for key in set(counts1.keys()) | set(counts2.keys())
    }

def is_common(value: t.Any) -> bool:
    return (type(value) == int or type(value) == bool) \
        and value in [0, 1, -1, True, False]

def is_axis(value: t.Any, max_input_rank) -> bool:
    return type(value) == int and value in range(2, max_input_rank + 1)

def is_shape(value: t.Any, dimension_lengths) -> bool:
    return type(value) == int and value in dimension_lengths

class CountConstantVisitor(ast.NodeVisitor):
    counts: ConstantCounts
    max_input_rank: int
    dimension_lengths: t.Set[int]
    output_shape: t.Optional[t.Tuple[int, ...]]

    should_print: bool

    def __init__(self, max_input_rank: int, dimension_lengths: t.Set[int], output_shape: t.Optional[t.Tuple[int, ...]] = None, should_print = False):
        self.counts = {
            "common": 0,
            "axis": 0,
            "shape": 0,
            "provided": 0,
            "tf_int32": 0,
            "tf_float32": 0,
            "tf_int64": 0,
            "tf_bool": 0,
            "input_var": 0,
            "shape_tuple": 0
        }
        self.max_input_rank = max_input_rank
        self.dimension_lengths = dimension_lengths
        self.output_shape = output_shape
        self.should_print = should_print

    def __print(self, *args):
        if self.should_print:
            print(*args)
    
    def visit_Constant(self, node: ast.Constant):
        if is_common(node.value):
            self.counts["common"] += 1
            self.__print("common", node.value)
        elif is_axis(node.value, self.max_input_rank):
            self.counts["axis"] += 1
            self.__print("axis", node.value)
        elif is_shape(node.value, self.dimension_lengths):
            self.counts["shape"] += 1
            self.__print("shape", node.value)
        elif type(node.value) == int:
            self.counts["provided"] += 1
            self.__print("provided", node.value)
        else:
            self.__print("unknown constant", node.value)

    def visit_UnaryOp(self, node: ast.UnaryOp):
        if isinstance(node.op, ast.USub) and isinstance(node.operand, ast.Constant):
            value = -node.operand.value
            if is_common(value):
                self.counts["common"] += 1
                self.__print("common", value)
            elif type(value) == int:
                self.counts["provided"] += 1
                self.__print("provided", value)
            else:
                self.__print("unknown constant", -node.operand.value)
        else:
            self.generic_visit(node)
    
    def visit_Name(self, node: Name) -> Any:
        if node.id.startswith("in") or node.id == 'tensor' or node.id == 'indices' or node.id == 'updates':
            self.counts["input_var"] += 1
            self.__print("input_var", node.id)
        else:
            self.__print("unknown name", node.id)
    
    def visit_Attribute(self, node: Attribute) -> Any:
        code = ast.unparse(node)
        if code == "tf.int32":
            self.counts["tf_int32"] += 1
            self.__print("tf.int32")
        elif code == "tf.float32":
            self.counts["tf_float32"] += 1
            self.__print("tf.float32")
        elif code == "tf.int64":
            self.counts["tf_int64"] += 1
            self.__print("tf.int64")
        elif code == "tf.bool":
            self.counts["tf_bool"] += 1
            self.__print("tf.bool")
        else:
            self.generic_visit(node.value)
    
    def visit_Tuple(self, node: ast.Tuple) -> Any:
        try:
            value = ast.literal_eval(node)
        except:
            value = None
        if value is not None and  len(value) == len(self.output_shape) and all([v == o for v, o in zip(value, self.output_shape)]):
            self.counts["shape_tuple"] += 1
            self.__print("shape_tuple", value)
        else:
            for elt in node.elts:
                self.generic_visit(elt)
        

def count_constants(ast_node: ast.Module, example: Example, should_print = False) -> ConstantCounts:
    visitor = CountConstantVisitor(example.max_input_rank, example.dimension_lengths, example.output_shape, should_print)
    visitor.visit(ast_node)
    return visitor.counts

for completion, ast_, example in zip(TEST_COMPLETIONS, TEST_ASTS, TEST_EXAMPLES):
    print(completion)
    print("max_input_rank", example.max_input_rank, "dimension_lengths", example.dimension_lengths, "output_shape", example.output_shape)
    pprint(count_constants(ast_, example, should_print=True))
    print()

return tf.stack([tf.math.bincount(in1), in1], axis=1)
max_input_rank 1 dimension_lengths {6} output_shape (6, 2)
unknown name tf
input_var in1
input_var in1
common 1
{'axis': 0,
 'common': 1,
 'input_var': 2,
 'provided': 0,
 'shape': 0,
 'shape_tuple': 0,
 'tf_bool': 0,
 'tf_float32': 0,
 'tf_int32': 0,
 'tf_int64': 0}

return tf.sparse.slice(in1, start=[0,0,0], size=[1,-1,-1])
max_input_rank 3 dimension_lengths {800, 2} output_shape (1, 2, 800)
unknown name tf
input_var in1
common 0
common 0
common 0
common 1
common -1
common -1
{'axis': 0,
 'common': 6,
 'input_var': 1,
 'provided': 0,
 'shape': 0,
 'shape_tuple': 0,
 'tf_bool': 0,
 'tf_float32': 0,
 'tf_int32': 0,
 'tf_int64': 0}

return tf.sparse.slice(in1, start=[3,800,2], size=[1,-1,-1])
max_input_rank 3 dimension_lengths {800, 2} output_shape (1, 2, 800)
unknown name tf
input_var in1
axis 3
shape 800
axis 2
common 1
common -1
common -1
{'axis': 2,
 'common': 3,
 'input_var': 1,
 'provided': 0,
 'shape': 1,
 'shape_tuple': 0,
 '

In [ ]:
COUNTSES = []
for ast_nodes, example in zip(ASTSES, EXAMPLES):
    if ast_nodes is None:
        COUNTSES.append(None)
        continue
    counts = []
    for ast_node in ast_nodes:
        if ast_node is None:
            counts.append(None)
            continue
        try:
            counts.append(count_constants(ast_node, example))
        except Exception as e:
            print("# ast_node")
            print(ast.unparse(ast_node))
            print("# example")
            print(example)
            print("# error")
            print(e)
            print(traceback.format_exc())
            print()
            counts.append(None)
    COUNTSES.append(counts)

AGGREGATE_COUNTS = [
    {
        "common": 0,
        "axis": 0,
        "shape": 0,
        "provided": 0,
        "tf_int32": 0,
        "tf_float32": 0,
        "tf_int64": 0,
        "tf_bool": 0,
        "input_var": 0,
        "shape_tuple": 0
    }
    for _ in COUNTSES
]

for idx, counts, aggregate_count in zip(range(len(COUNTSES)), COUNTSES, AGGREGATE_COUNTS):
    if counts is None:
        continue
    for count in counts:
        if count is None:
            continue
        aggregate_count = add_constant_counts(aggregate_count, count)
        AGGREGATE_COUNTS[idx] = aggregate_count

print("COUNTSES", len(COUNTSES), "AGGREGATE_COUNTS", len(AGGREGATE_COUNTS), "EXAMPLES", len(EXAMPLES))
pprint(AGGREGATE_COUNTS[0])
pprint(COUNTSES[0])


COUNTSES 72 AGGREGATE_COUNTS 72 EXAMPLES 72
{'axis': 0,
 'common': 21,
 'input_var': 31,
 'provided': 4,
 'shape': 0,
 'shape_tuple': 0,
 'tf_bool': 0,
 'tf_float32': 0,
 'tf_int32': 3,
 'tf_int64': 0}
[{'axis': 0,
  'common': 1,
  'input_var': 2,
  'provided': 0,
  'shape': 0,
  'shape_tuple': 0,
  'tf_bool': 0,
  'tf_float32': 0,
  'tf_int32': 0,
  'tf_int64': 0},
 {'axis': 0,
  'common': 3,
  'input_var': 4,
  'provided': 0,
  'shape': 0,
  'shape_tuple': 0,
  'tf_bool': 0,
  'tf_float32': 0,
  'tf_int32': 0,
  'tf_int64': 0},
 {'axis': 0,
  'common': 1,
  'input_var': 2,
  'provided': 2,
  'shape': 0,
  'shape_tuple': 0,
  'tf_bool': 0,
  'tf_float32': 0,
  'tf_int32': 2,
  'tf_int64': 0},
 {'axis': 0,
  'common': 1,
  'input_var': 2,
  'provided': 2,
  'shape': 0,
  'shape_tuple': 0,
  'tf_bool': 0,
  'tf_float32': 0,
  'tf_int32': 0,
  'tf_int64': 0},
 {'axis': 0,
  'common': 3,
  'input_var': 4,
  'provided': 0,
  'shape': 0,
  'shape_tuple': 0,
  'tf_bool': 0,
  'tf_float32': 0

In [ ]:
for counts, aggregate_count, output in zip(COUNTSES, AGGREGATE_COUNTS, OUTPUTS_JSON):
    if 'response' not in output:
        continue
    output['response']['constant_counts'] = counts
    output['response']['aggregate_constant_count'] = aggregate_count

pprint(OUTPUTS_JSON[0])

{'completions': ['return tf.stack([tf.math.bincount(in1), in1], axis=1)',
                 'return tf.stack([tf.divide(in1, tf.shape(in1)[0]), '
                 'tf.mod(in1, tf.shape(in1)[0])], axis=1)',
                 'return tf.stack([tf.math.floor_div(in1, tf.constant(2, '
                 'dtype=tf.int32)), tf.math.floormod(in1, tf.constant(2, '
                 'dtype=tf.int32))], axis=1)',
                 'return tf.stack([tf.floor_div(in1, 2), tf.mod(in1, 2)], '
                 'axis=1)',
                 'return tf.stack([tf.math.floor_divide(in1, '
                 'tf.shape(in1)[0]), tf.math.mod(in1, tf.shape(in1)[0])], '
                 'axis=1)',
                 'return tf.stack([tf.math.floor_divide(in1, '
                 'tf.reduce_max(in1) + 1), tf.math.floormod(in1, '
                 'tf.reduce_max(in1) + 1)], axis=1)',
                 'return tf.stack([tf.math.floor_div(in1, tf.reduce_max(in1) + '
                 '1), tf.math.floormod(in1, tf.reduce_max(in1)

In [ ]:
OUTPUT_FILE = CURRENT_DIRECTORY / "tfcoder_dataset_with_completions_constants_and_counts.json"
OUTPUT_FILE.write_text(json.dumps(OUTPUTS_JSON, indent=4))

549631